# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Входные данные — персональные данные клиентов находятся в файле `insurance.csv`. О качестве данных ничего не известно, по-этому перед исследованием понадобится обзор данных.

*Описание данных:*

 - **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
 - **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.
 
**Разработка пройдёт в четыре основных этапа:**

1. Загрузка и подготовка данных.
2. Умножение матриц.
3. Алгоритм преобразования.
4. Проверка алгоритма. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Загрузка данных   
Загрузим необходимые для нас библиотеки.

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

pd.options.display.float_format = '{:,.3f}'.format

print('Setup complete')

Setup complete


### Обзор данных   
Теперь загрузим и изучим данные 

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('datasets/insurance.csv')

In [3]:
def get_info(df):
    display(df.head(), df.info(), df.describe().T)
    print('Процент пропусков:\n', df.isna().mean()*100)
    print()
    print('Кол-во дубликатов:', df.duplicated().sum())

In [4]:
get_info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.000,"49,600.000",1,0
1,0,46.000,"38,000.000",1,1
2,0,29.000,"21,000.000",0,0
3,0,21.000,"41,700.000",2,0
4,1,28.000,"26,100.000",0,0


None

,count,mean,std,min,25%,50%,75%,max
Пол,"5,000.000",0.499,0.500,0.000,0.000,0.000,1.000,1.000
Возраст,"5,000.000",30.953,8.441,18.000,24.000,30.000,37.000,65.000
Зарплата,"5,000.000","39,916.360","9,900.084","5,300.000","33,300.000","40,200.000","46,600.000","79,000.000"
Члены семьи,"5,000.000",1.194,1.091,0.000,0.000,1.000,2.000,6.000
Страховые выплаты,"5,000.000",0.148,0.463,0.000,0.000,0.000,0.000,5.000


Процент пропусков:
 Пол                 0.000
Возраст             0.000
Зарплата            0.000
Члены семьи         0.000
Страховые выплаты   0.000
dtype: float64

Кол-во дубликатов: 153


Здесь сразу сделаем выводы что следует удалить дубликаты и заменить наименования столбцов. А так же можно привести все признаки к целочисленному формату данных. Детальнее изучать данные не будем, так как целью разработки не является получения качественной модели.

### Предобработка данных

In [5]:
df.drop_duplicates(inplace=True)
df = df.astype('int')
df.columns = ['gender', 'age', 'salary', 'family', 'ins_pay']
print('Кол-во дубликатов:', df.duplicated().sum())
df.info()

Кол-во дубликатов: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   gender   4847 non-null   int64
 1   age      4847 non-null   int64
 2   salary   4847 non-null   int64
 3   family   4847 non-null   int64
 4   ins_pay  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


Дубликатов больше нет, формат данных целочисленный, имена столбцов корректны, считаем что предобработка выполнена.

## Умножение матриц

Ответим на вопрос и обоснуем решение.   
Признаки умножим на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$



**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** Для обоснования нам необходимо доказать, что предсказания $a_1$ - предсказания до преобразования равны $a_2$ - предсказания после преобразования.   

Запишем формулу предсказания после преобразования:

$$
a_2 = PXw_2
$$

Отсюда можно подставить умножение матрицы признаков на обратимую матрицу $P$ в формулу обучения:

$$
w_2 = ((PX)^T PX)^{-1} (PX)^T y
$$

Обратная для квадратной матрицы A — матрица A с верхним индексом $^{-1}$, произведение на А равно единичной матрице.

$$
AA^{-1} = A^{-1}A = E
$$

Единичная матрица (E) — это квадратная матрица, на главной диагонали которой стоят единицы, а остальные элементы — нули. Если любую матрицу A умножить на единичную (или наоборот), получится эта же матрица A:

$$
AE = EA = A
$$

Далее вынесем за скобки матрицу $P$ в формуле обучения:

$$
w_2 = (P^T)^{-1} P^T (X^T X)^{-1} P^{-1} X^T y
$$

Преобразуем формулу убрав произведение матрицы на обратную к ней:

$$
w_2 = E (X^T X)^{-1} P^{-1} X^T y = (X^T X)^{-1} P^{-1} X^T y
$$

Часть получившейся формулы можно преобразовать в формулу обучения без преобразования:

$$
w_2 = (X^T X)^{-1} X^T y P^{-1} = w_1 P^{-1}
$$

Подставим значения $w_2$ в формулу предсказания после преобразования

$$
a_2 = PXw_2 = PP^{-1}Xw_1 = EXw_1 = Xw_1 = a_1
$$

Как мы видим, предсказания не изменятся после преобразования признаков умножением на обратимую матрицу, следовательно и качество предсказания останется таким же.

## Алгоритм преобразования

**Алгоритм**

Напишем алгоритм, по которому будем преобразовывать наши признаки:

1. Умножение признаков на квадратную матрицу размером (4,4)
2. Прибавим к каждому признаку константу 101
3. Обучаем модель на новых признаках
4. Получаем предсказания
5. В качестве метрики берем коэффициент детерминации R2;
6. Обучаем модель на исходных данных и получаем предсказания на исходных данных;
7. Сравниваем два коэффициента детерминации и определяем работоспособность алгоритма   

Выделим целевой признак и преобразуем остальные согласно алгоритма.

In [6]:
df.head()

,gender,age,salary,family,ins_pay
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [7]:
target = df['ins_pay'].values
features = df.drop('ins_pay', axis=1).values
features

array([[    1,    41, 49600,     1],
       [    0,    46, 38000,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33900,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40600,     1]])

Создадим случайную матрицу и проверим её на обратимость.

In [8]:
rnd_matrix = np.random.rand(4,4)
np.linalg.inv(rnd_matrix)

array([[ -1.09500674, -13.50299747,  26.19134578, -13.67976064],
       [  2.14878045,   6.7879244 , -14.35906751,   7.42456828],
       [  1.36710625,  16.7912482 , -28.06644095,  12.8666713 ],
       [ -0.87274676,   1.13254948,  -1.97326353,   2.37052627]])

Напишем фунцию для преобразования признаков

In [9]:
def coding(X):
    X = X @ rnd_matrix
    X += 101
    
    return X

In [10]:
new_features = coding(features)

In [11]:
new_features

array([[35592.00357136, 25162.4546823 , 15705.97766574, 41718.27085212],
       [27302.55618009, 19306.01388758, 12063.46766061, 31995.05752623],
       [15135.94238347, 10715.26928073,  6713.54114588, 17728.58469073],
       ...,
       [24351.9140883 , 17227.00320006, 10762.5509632 , 28540.61228185],
       [23496.76426597, 16623.14111104, 10386.78956772, 27537.02449922],
       [29148.01498572, 20613.27841479, 12871.75057647, 34163.20451201]])

Выглядет не узнаваемо.   
Напишем функцию декодирования

**Обоснование**

In [12]:
def decoding(X):
    X -= 101
    X = X @ np.linalg.inv(rnd_matrix)
    X = X.astype('int')
    
    return X

In [13]:
old_features = decoding(new_features)
old_features

array([[    1,    40, 49600,     0],
       [    0,    46, 37999,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33899,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40599,     0]])

Вернулись к первичному виду признаков.   
Здесь можно сказать что повторяется формула
$$
a_2 = PXw_2 = PP^{-1}Xw_1 = EXw_1 = Xw_1 = a_1
$$

Напишем функцию для получения метрики.

In [14]:
def get_r2_lr_valid(X, y, ratio = 0.8):
    
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    
    # Classifier training
    lr = LinearRegression().fit(X[:idx, :], y[:idx])
    
    # Prediction for validation set
    y_pred = lr.predict(X[idx:, :])#[:, 1]
    
    # Calculate the quality
    score = r2_score(y[idx:], y_pred)
    
    return score

## Проверка алгоритма   
Проверим наш алгоритм в деле на двух видах признаков.   
Сравним показатели детерминации R2 на признаках до и после преобразования.

In [15]:
get_r2_lr_valid(features, target)

0.4461776331245556

In [16]:
get_r2_lr_valid(new_features, target)

0.44617763312527214

## Общий вывод

Перед нами стояла задача защитить данные клиентов страховой компании «Хоть потоп». Мы разработали такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Потери качества пресказания после преобразования признаков не происходит, показатели детерминации R2 на признаках до (0.44617763312455805) и после (0.4461776331237476) преобразования практически не отличаются.